Imports and Setup

In [20]:

import pandas as pd
import numpy as np
import os
import pickle
import gzip
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score


 Load and Inspect the Dataset

In [21]:
# Load the dataset
data_path = 'data_new.csv'
if not os.path.exists(data_path):
    raise FileNotFoundError(f"The data file {data_path} does not exist.")

df = pd.read_csv(data_path)

# Display the first few rows
df.head()


,risk_score_t,program_enrolled_t,cost_t,cost_avoidable_t,bps_mean_t,ghba1c_mean_t,hct_mean_t,cre_mean_t,ldl_mean_t,race,...,trig_min-high_tm1,trig_min-normal_tm1,trig_mean-low_tm1,trig_mean-high_tm1,trig_mean-normal_tm1,trig_max-low_tm1,trig_max-high_tm1,trig_max-normal_tm1,gagne_sum_tm1,gagne_sum_t
0,1.987430,0,1200.0,0.0,NaN,5.4,NaN,1.110000,194.0,white,...,0,0,0,0,0,0,0,0,0,0
1,7.677934,0,2600.0,0.0,119.0,5.5,40.4,0.860000,93.0,white,...,0,1,0,0,1,0,0,1,4,3
2,0.407678,0,500.0,0.0,NaN,NaN,NaN,NaN,NaN,white,...,0,0,0,0,0,0,0,0,0,0
3,0.798369,0,1300.0,0.0,117.0,NaN,NaN,NaN,NaN,white,...,0,0,0,0,0,0,0,0,0,0
4,17.513165,0,1100.0,0.0,116.0,NaN,34.1,1.303333,53.0,white,...,0,0,0,0,0,0,0,0,1,1


Data Preprocessing

In [22]:
# Define age band columns and their corresponding numerical values
age_band_columns = [
    'dem_age_band_18-24_tm1', 'dem_age_band_25-34_tm1', 'dem_age_band_35-44_tm1',
    'dem_age_band_45-54_tm1', 'dem_age_band_55-64_tm1', 'dem_age_band_65-74_tm1', 'dem_age_band_75+_tm1'
]

age_mapping = {
    'dem_age_band_18-24_tm1': 21,
    'dem_age_band_25-34_tm1': 30,
    'dem_age_band_35-44_tm1': 40,
    'dem_age_band_45-54_tm1': 50,
    'dem_age_band_55-64_tm1': 60,
    'dem_age_band_65-74_tm1': 70,
    'dem_age_band_75+_tm1': 80
}

# Initialize 'age' with zeros
df['age'] = 0

# Assign age values based on the presence in each band
for band, age_val in age_mapping.items():
    if band in df.columns:
        df['age'] += df[band] * age_val


# Handle missing values by filling with median for numeric columns
df.fillna(df.median(numeric_only=True), inplace=True)

# Encode the categorical variable 'race' if it is non-numeric
if 'race' in df.columns and df['race'].dtype == 'object':
    df['race'] = LabelEncoder().fit_transform(df['race'])


# Define biomarker columns
biomarker_columns = [
    'cre_min-low_tm1', 'cre_min-high_tm1', 'cre_min-normal_tm1',
    'cre_mean-low_tm1', 'cre_mean-high_tm1', 'cre_mean-normal_tm1',
    'cre_max-low_tm1', 'cre_max-high_tm1', 'cre_max-normal_tm1',
    'crp_min-low_tm1', 'crp_min-high_tm1', 'crp_min-normal_tm1',
    'crp_mean-low_tm1', 'crp_mean-high_tm1', 'crp_mean-normal_tm1',
    'crp_max-low_tm1', 'crp_max-high_tm1', 'crp_max-normal_tm1',
    'esr_min-low_tm1', 'esr_min-high_tm1', 'esr_min-normal_tm1',
    'esr_mean-low_tm1', 'esr_mean-high_tm1', 'esr_mean-normal_tm1',
    'esr_max-low_tm1', 'esr_max-high_tm1', 'esr_max-normal_tm1',
    'ghba1c_min-low_tm1', 'ghba1c_min-high_tm1', 'ghba1c_min-normal_tm1',
    'ghba1c_mean-low_tm1', 'ghba1c_mean-high_tm1', 'ghba1c_mean-normal_tm1',
    'ghba1c_max-low_tm1', 'ghba1c_max-high_tm1', 'ghba1c_max-normal_tm1',
    'hct_min-low_tm1', 'hct_min-high_tm1', 'hct_min-normal_tm1',
    'hct_mean-low_tm1', 'hct_mean-high_tm1', 'hct_mean-normal_tm1',
    'hct_max-low_tm1', 'hct_max-high_tm1', 'hct_max-normal_tm1',
    'ldl_min-low_tm1', 'ldl_min-high_tm1', 'ldl_min-normal_tm1',
    'ldl-mean-low_tm1', 'ldl-mean-high_tm1', 'ldl-mean-normal_tm1',
    'ldl_max-low_tm1', 'ldl_max-high_tm1', 'ldl_max-normal_tm1',
    'nt_bnp_min-low_tm1', 'nt_bnp_min-high_tm1', 'nt_bnp_min-normal_tm1',
    'nt_bnp_mean-low_tm1', 'nt_bnp_mean-high_tm1', 'nt_bnp_mean-normal_tm1',
    'nt_bnp_max-low_tm1', 'nt_bnp_max-high_tm1', 'nt_bnp_max-normal_tm1',
    'sodium_min-low_tm1', 'sodium_min-high_tm1', 'sodium_min-normal_tm1',
    'sodium_mean-low_tm1', 'sodium_mean-high_tm1', 'sodium_mean-normal_tm1',
    'sodium_max-low_tm1', 'sodium_max-high_tm1', 'sodium_max-normal_tm1',
    'trig_min-low_tm1', 'trig_min-high_tm1', 'trig_min-normal_tm1',
    'trig_mean-low_tm1', 'trig_mean-high_tm1', 'trig_mean-normal_tm1',
    'trig_max-low_tm1', 'trig_max-high_tm1', 'trig_max-normal_tm1'
]

# Select columns where data is present (non-null values)
non_empty_biomarker_columns = df[biomarker_columns].dropna(axis=1, how='all')

# Randomly select 10 columns with data present
if len(non_empty_biomarker_columns.columns) < 10:
    random_10_biomarker_columns = non_empty_biomarker_columns.columns.tolist()
    print(f"Only {len(random_10_biomarker_columns)} biomarker columns available. Using all.")
else:
    random_10_biomarker_columns = non_empty_biomarker_columns.sample(n=10, axis=1, random_state=42).columns.tolist()

# Ensure selected biomarker columns exist in the DataFrame
valid_biomarker_columns = [col for col in random_10_biomarker_columns if col in df.columns]

# Create the 'biomarkers' column
if valid_biomarker_columns:
    # Example consolidation: set to 1 if any 'normal' biomarker is > 0
    df['biomarkers'] = df[valid_biomarker_columns].apply(
        lambda row: 1 if any('normal' in col and row[col] > 0 for col in valid_biomarker_columns) else 0,
        axis=1
    )
else:
    df['biomarkers'] = 0
    print("No valid biomarker columns found in the dataset. Setting 'biomarkers' to 0.")


# Define comorbidity columns
comorbidity_columns = [
    'alcohol_elixhauser_tm1', 'anemia_elixhauser_tm1', 'arrhythmia_elixhauser_tm1',
    'arthritis_elixhauser_tm1', 'bloodlossanemia_elixhauser_tm1', 'coagulopathy_elixhauser_tm1',
    'compdiabetes_elixhauser_tm1', 'depression_elixhauser_tm1', 'drugabuse_elixhauser_tm1',
    'electrolytes_elixhauser_tm1', 'hypertension_elixhauser_tm1', 'hypothyroid_elixhauser_tm1',
    'liver_elixhauser_tm1', 'neurodegen_elixhauser_tm1', 'obesity_elixhauser_tm1',
    'paralysis_elixhauser_tm1', 'psychosis_elixhauser_tm1', 'pulmcirc_elixhauser_tm1',
    'pvd_elixhauser_tm1', 'renal_elixhauser_tm1', 'uncompdiabetes_elixhauser_tm1',
    'valvulardz_elixhauser_tm1', 'wtloss_elixhauser_tm1', 'cerebrovasculardz_romano_tm1',
    'chf_romano_tm1', 'dementia_romano_tm1', 'hemiplegia_romano_tm1', 'hivaids_romano_tm1',
    'metastatic_romano_tm1', 'myocardialinfarct_romano_tm1', 'pulmonarydz_romano_tm1',
    'tumor_romano_tm1', 'ulcer_romano_tm1'
]

# Select columns where data is present (non-null values)
non_empty_comorbidity_columns = df[comorbidity_columns].dropna(axis=1, how='all')

# Randomly select 10 columns with data present
if len(non_empty_comorbidity_columns.columns) < 10:
    random_10_comorbidity_columns = non_empty_comorbidity_columns.columns.tolist()
    print(f"Only {len(random_10_comorbidity_columns)} comorbidity columns available. Using all.")
else:
    random_10_comorbidity_columns = non_empty_comorbidity_columns.sample(n=10, axis=1, random_state=42).columns.tolist()

# Ensure selected comorbidity columns exist in the DataFrame
valid_comorbidity_columns = [col for col in random_10_comorbidity_columns if col in df.columns]

# Create the 'comorbidity' column
if valid_comorbidity_columns:
    # Example consolidation: set to 1 if any comorbidity is present
    df['comorbidity'] = df[valid_comorbidity_columns].apply(
        lambda row: 1 if any(row[col] > 0 for col in valid_comorbidity_columns) else 0,
        axis=1
    )
else:
    df['comorbidity'] = 0
    print("No valid comorbidity columns found in the dataset. Setting 'comorbidity' to 0.")



Feature and Target Selection

In [23]:
# Define the features to be used for prediction
features = [
    'age', 'dem_female', 'race', 'biomarkers', 'comorbidity',
    'lasix_dose_count_tm1', 'cre_tests_tm1', 'crp_tests_tm1', 'esr_tests_tm1',
    'ghba1c_tests_tm1', 'hct_tests_tm1', 'ldl_tests_tm1', 'nt_bnp_tests_tm1',
    'sodium_tests_tm1', 'trig_tests_tm1'
]

# Check that all features are available in the dataset
missing_features = [feature for feature in features if feature not in df.columns]
if missing_features:
    print(f"Missing features from the dataset: {missing_features}")


# Define the target variables
target_columns = ['risk_score_t', 'cost_t', 'bps_mean_t', 'gagne_sum_t', 'ldl_mean_t']

# Check that all target columns exist
missing_targets = [target for target in target_columns if target not in df.columns]
if missing_targets:
    print(f"Missing target columns from the dataset: {missing_targets}")
    raise KeyError(f"The following target columns are missing: {missing_targets}")


Model Training and Evaluation

In [24]:
# Proceed only if no features or targets are missing
if not missing_features and not missing_targets:
    # Prepare the features (X) and targets (y)
    X = df[features]
    y = df[target_columns]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Standardize the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Save the scaler
    with open('standard_scaler.pkl', 'wb') as scaler_file_out:
        pickle.dump(scaler, scaler_file_out)

    # Compress the scaler
    with open('standard_scaler.pkl', 'rb') as scaler_file_in:
        with gzip.open('standard_scaler.pkl.gz', 'wb') as scaler_file_out_compressed:
            scaler_file_out_compressed.write(scaler_file_in.read())

    # Initialize and train the MultiOutput Random Forest model
    model = MultiOutputRegressor(RandomForestRegressor(n_estimators=100, random_state=42))
    model.fit(X_train_scaled, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test_scaled)

    # Evaluate the model for each target and get feature importance
for idx, target in enumerate(target_columns):
    # Calculate evaluation metrics
    mse = mean_squared_error(y_test.iloc[:, idx], y_pred[:, idx])
    r2 = r2_score(y_test.iloc[:, idx], y_pred[:, idx])
    print(f"--- {target} ---")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}\n")
    
    # Get feature importance for the current target
    feature_importance = model.estimators_[idx].feature_importances_
    feature_importance_df = pd.DataFrame({
        'Feature': features,
        'Importance': feature_importance
    }).sort_values(by='Importance', ascending=False)
    
    print(f"Feature Importance for '{target}':")
    print(feature_importance_df)
    print("\n")


    # Save the multi-output random forest model
    with open('multi_output_random_forest_model.pkl', 'wb') as model_file_out:
        pickle.dump(model, model_file_out)

    # Compress the model
    with open('multi_output_random_forest_model.pkl', 'rb') as model_file_in:
        with gzip.open('multi_output_random_forest_model_compressed.pkl.gz', 'wb') as model_file_out_compressed:
            model_file_out_compressed.write(model_file_in.read())

    print("Model training and evaluation completed successfully.")
else:
    print("Cannot proceed with model training due to missing features or target columns.")


--- risk_score_t ---
Mean Squared Error: 18.927368668148475
R-squared: 0.3315504066485432

Feature Importance for 'risk_score_t':
                 Feature  Importance
6          cre_tests_tm1    0.331480
0                    age    0.116470
10         hct_tests_tm1    0.110542
4            comorbidity    0.088552
13      sodium_tests_tm1    0.082573
9       ghba1c_tests_tm1    0.053679
8          esr_tests_tm1    0.041045
14        trig_tests_tm1    0.037929
12      nt_bnp_tests_tm1    0.031657
11         ldl_tests_tm1    0.030893
1             dem_female    0.029604
2                   race    0.023662
5   lasix_dose_count_tm1    0.021110
7          crp_tests_tm1    0.000805
3             biomarkers    0.000000


Model training and evaluation completed successfully.
--- cost_t ---
Mean Squared Error: 318659790.500639
R-squared: 0.014538118619836782

Feature Importance for 'cost_t':
                 Feature  Importance
6          cre_tests_tm1    0.174385
0                    age    0.

Interactive Prediction

In [27]:
# Load the scaler
with open('standard_scaler.pkl', 'rb') as scaler_file_in:
    scaler = pickle.load(scaler_file_in)

# Load the model
with open('multi_output_random_forest_model.pkl', 'rb') as model_file_in:
    model = pickle.load(model_file_in)


# Define a sample input DataFrame for prediction
sample_input = pd.DataFrame({
    'age': [57],               # Example age
    'dem_female': [0],         # Male
    'race': [1],               # Encoded race
    'biomarkers': [1],         # Biomarker status
    'comorbidity': [1],        # Comorbidity status
    'lasix_dose_count_tm1': [0],   # Example value
    'cre_tests_tm1': [100],         # Example value
    'crp_tests_tm1': [2],           # Example value
    'esr_tests_tm1': [1],           # Example value
    'ghba1c_tests_tm1': [3],        # Example value
    'hct_tests_tm1': [1],           # Example value
    'ldl_tests_tm1': [2],           # Example value
    'nt_bnp_tests_tm1': [0],        # Example value
    'sodium_tests_tm1': [1],        # Example value
    'trig_tests_tm1': [2]           # Example value
})

print(sample_input)

# Specify desired targets for prediction
# Choices are: 'risk_score_t', 'cost_t','bps_mean_t', 'gagne_sum_t'

desired_targets = ['bps_mean_t']  # Example: Predict only 'risk_score_t'

# Validate desired targets
valid_targets = [target for target in desired_targets if target in target_columns]
invalid_targets = [target for target in desired_targets if target not in target_columns]

if invalid_targets:
    print(f"Invalid target(s) specified: {invalid_targets}")
    print(f"Valid targets are: {target_columns}")
else:
    print(f"Selected target(s) for prediction: {valid_targets}")

# Define feature importances for each target (this should be based on your actual model)


   age  dem_female  race  biomarkers  comorbidity  lasix_dose_count_tm1  \
0   57           0     1           1            1                     0   

   cre_tests_tm1  crp_tests_tm1  esr_tests_tm1  ghba1c_tests_tm1  \
0            100              2              1                 3   

   hct_tests_tm1  ldl_tests_tm1  nt_bnp_tests_tm1  sodium_tests_tm1  \
0              1              2                 0                 1   

   trig_tests_tm1  
0               2  
Selected target(s) for prediction: ['bps_mean_t']


Make Predictions Based on Selected Targets

In [28]:
if not invalid_targets:
    # Ensure the sample input has all required features
    if all(feature in sample_input.columns for feature in features):
        # Scale the sample input
        X_sample_scaled = scaler.transform(sample_input[features])

        # Make predictions
        predicted_values = model.predict(X_sample_scaled)[0]
        prediction_series = pd.Series(predicted_values, index=target_columns)

        # Select only the desired targets
        selected_predictions = prediction_series[valid_targets]

        print(selected_predictions)

    else:
        missing_in_input = [feature for feature in features if feature not in sample_input.columns]
        print(f"Sample input is missing required features: {missing_in_input}")


bps_mean_t    132.333333
dtype: float64
